## 该代码提供方法示例，涉及到数据安全，所以不代表最终的实现。请供参考。


In [ ]:
import csv
import faiss
import openai
import numpy as np
import pandas as pd

graph_path = 'your graph.csv'
chunks_path = 'your chunks.csv'
# 读取 graph.csv 文件
graph_df = pd.read_csv(graph_path, delimiter='|', names=['node_1', 'node_2', 'edge', 'chunk_id'])

# 读取 chunks.csv 文件
chunks_df = pd.read_csv(chunks_path, delimiter='|', names=['text', 'source', 'chunk_id'])

graph_df

In [ ]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings as BaseEmbeddings
from typing import List

class SentenceTransformerEmbeddings(BaseEmbeddings):
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, convert_to_tensor=False).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode([text], convert_to_tensor=False).tolist()[0]


embedding_model = SentenceTransformerEmbeddings(model_name='your model name')
embedding = embedding_model

In [ ]:
# 为每一行生成Embedding并存储到列表
embeddings = []
triplets = []
for index, row in graph_df.iterrows():
    triplet_text = f"({row['node_1']} , {row['edge']} , {row['node_2']})"
    triplet_embedding = embedding_model.embed_query(triplet_text)
    embeddings.append((triplet_embedding, row['chunk_id']))
    triplets.append(triplet_text)


In [ ]:
# 将生成的向量存储到FAISS向量数据库
dimension = len(embeddings[0][0])
index = faiss.IndexFlatL2(dimension)

# 存储chunk_id的列表
chunk_ids = []

for embedding, chunk_id in embeddings:
    embedding_array = np.array(embedding, dtype=np.float32).reshape(1, -1)  # 确保是float32类型的二维数组
    index.add(embedding_array)
    chunk_ids.append(chunk_id)

In [ ]:
# 保存索引和chunk_id
faiss.write_index(index, 'vector_index.faiss')
with open('chunk_ids.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows([[cid] for cid in chunk_ids])

Agent

In [ ]:
import faiss
import numpy as np
import csv
import os
import pandas as pd

# 检查是否已经存在保存的索引和chunk_id文件
index_file = 'vector_index.faiss'
chunk_ids_file = 'chunk_ids.csv'

def load_or_create_index():
    if os.path.exists(index_file) and os.path.exists(chunk_ids_file):
        # 如果文件存在，直接读取
        index = faiss.read_index(index_file)
        with open(chunk_ids_file, 'r') as f:
            reader = csv.reader(f)
            next(reader)  # 跳过第一行（列名）
            chunk_ids = [row[0] for row in reader]  # 直接读取字符串
        triplets = []
        for indexs,row in graph_df.iterrows():
            triplet_text = f"({row['node_1']} , {row['edge']} , {row['node_2']})"
            triplets.append(triplet_text)
    else:
        # 如果文件不存在，生成并保存嵌入向量和chunk_id
        embeddings = []
        triplets = []
        for index, row in graph_df.iterrows():
            triplet_text = f"({row['node_1']} , {row['edge']} , {row['node_2']})"
            triplet_embedding = embedding_model.embed_query(triplet_text)
            embeddings.append((triplet_embedding, row['chunk_id']))
            triplets.append(triplet_text)

        # 将生成的向量存储到FAISS向量数据库
        dimension = len(embeddings[0][0])
        index = faiss.IndexFlatL2(dimension)

        # 存储chunk_id的列表
        chunk_ids = []

        for embedding, chunk_id in embeddings:
            embedding_array = np.array(embedding, dtype=np.float32).reshape(1, -1)  # 确保是float32类型的二维数组
            index.add(embedding_array)
            chunk_ids.append(str(chunk_id))  # 确保chunk_id是字符串

        # 保存索引和chunk_id
        faiss.write_index(index, index_file)
        with open(chunk_ids_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['chunk_id'])  # 写入列名
            writer.writerows([[cid] for cid in chunk_ids])
    
    return index, chunk_ids,triplets

# 加载或创建索引
index, chunk_ids ,triplets= load_or_create_index()

# 步骤2: 向量相似度查询
def query_database(querys, index=index, chunk_ids=chunk_ids,triplets=triplets):
    unique_triplets = set()
    unique_chunk_ids = set()
    for query in querys:
        query_embedding = embedding_model.embed_query(query)
        query_embedding_array = np.array(query_embedding, dtype=np.float32).reshape(1, -1)  # 确保是float32类型的二维数组
        D, I = index.search(query_embedding_array, k=5)  # 找到最相似的5个向量
        similar_chunk_ids = [chunk_ids[i] for i in I[0] if i < len(chunk_ids)]  # 检查索引有效性
        unique_chunk_ids.update(similar_chunk_ids)                                       

        # 步骤4: 查找对应的三元组并添加到集合
        matching_triplets = [triplets[i] for i in I[0] if i < len(triplets)]  # 检查索引有效性
        unique_triplets.update(matching_triplets)

    matching_chunks = chunks_df[chunks_df['chunk_id'].isin(unique_chunk_ids)]['text']

    return unique_triplets, matching_chunks

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
#可替换为本地模型
os.environ['OPENAI_API_KEY'] = "your api key"
os.environ['OPENAI_API_BASE'] = "your api base"
llm = ChatOpenAI(model='local_model or online_model')

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
import re
def rewrite_and_split_query(query):
    # 使用智能体来重写和拆分查询
    prompt = PromptTemplate(
        input_variables=["query"],
        template="""
        你是个智能体助手，我给你给你一个query，你需要把问题拆解为便于用于在知识图谱向量数据库中查询的短语，提升在向量数据库中匹配的准确率。
        你只需要回答出拆解后的短语即可，不要添加或者修改，中文回答。
        接下来给你一个例子：
        例子1:
            问题:
                在直动式顺序阀是如何通过控制油液压力来实现缸Ⅰ和缸Ⅱ按顺序动作的？
            短语: 
                直通式顺序阀\n
                油液压力\n
                缸Ⅰ和缸Ⅱ按顺序动作\n
        例子2：
            问题:
                在液压系统中，如何通过故障诊断来确定并解决‘系统压力波动’的问题？
            短语：
                液压系统\n
                故障诊断\n
                系统压力波动\n
        请把下面的问题拆解为短语，你必须严格按照上述规定的短语格式输出。
        问题: {query}
        短语:
        """
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    result = chain.run(query)
    sub_queries = result.split('\n')  # 假设结果是按行分隔的
    sub_queries.append(query)  # 最后一行是query本身
    return sub_queries

# 步骤2: 优化查询结果并回答
def optimize_contexts(query,contexts):
    # 利用智能体优化三元组和chunks
    prompt = PromptTemplate(
        input_variables=["query, contexts"],
        template="""
        你是一个智能体助手，我给你一个query和一些相关的上下文contexts,由于上下文过长，你需要从中提炼出对于解答query相关的，直接的，有用的信息。
        query: {query}
        contexts: {contexts}
        首先你需要对query进行分析，明确query的意图，以及分析出解答query所需要的可能的信息。
        然后需要对contexts进行分析，提炼出所有有用的信息，最终输出一个优化的上下文，用来解答query。输出的结果需要包含三元组和context，确保这些三元组和context是与query相关的。
        你需要输出三元组及其对三元组的解释，以及context。
        你只需要提炼出与query相关的信息！！！尽量保持原文信息一致性，不用描述和概括。
        不允许出现英文回答，只能用中文回答。
        你的输出必须符合下述格式！不允许输出别的内容。
        请一句一句分析，不能跳过任何一句，一步一步思考。
        输出格式如下：
        Triplets:
        (node1,relation,node2)，后面是对形成三元组和解答query有用的信息的解释。
        Context: 
        优化提炼之后的上下文。
        请开始提炼:
        """
    )
    input = {
        "query": query,
        "contexts": contexts
    }
    chain = LLMChain(llm=llm, prompt=prompt)
    optimized_contexts = chain.run(input)
    
    return optimized_contexts

# 步骤3: 思考，纠正，再次优化
def think_And_reflect(query,contexts,n=3):
    # 利用智能体优化三元组和chunks
    prompt = PromptTemplate(
        input_variables=["query, contexts, n"],
        template="""
        你是一个世界级的人工智能系统，能够进行复杂且准确的推理和反思，经过{n}次迭代的thinking和reflection，最终得到[answer]。
        query: {query}
        contexts: {contexts}
        首先在[thinking]标签下进行推理，然后在[reflection]标签反思你的回答，分析可能出现的错误和能够改进的地方，并重复这个过程，反复进行{n}轮的[thinking]和[reflection]。最后，你得到了[answer]。
        请注意，我提供的contexts里面包含三元组Triplets以及context内容，内容可能和query相关，但不一定是直接的答案。所以你需要一一思考，反思，再次优化，最终得到一个优化的答案。
        你的分析过程和最终answer必须来自于contexts，不能捏造和歪曲！
        请一步一步认真思考，最后结果用中文输出。
        你必须按照以下格式输出！
        输出格式如下：
        第一轮迭代:
        [thinking]
        你的推理过程。
        [reflection]
        你的反思过程。
        第二轮迭代:
        [thinking]
        你的推理过程。
        [reflection]
        你的反思过程。
        ......
        [answer]：
        你最终的答案，你只需要回答问题的答案即可，不用添加其他描述，例如“根据上下文”、“根据以上分析”等语句。
        确保要回答到问题中的每个要点，答案不需要分点回答。要表现的像一个标准答案。
        接下来，请一步一步思考，并且必须按照格式输出！回答的最后一定要包含[answer]！
        """
    )
    input = {
        "query": query,
        "contexts": contexts,
        "n": n
    }
    chain = LLMChain(llm=llm, prompt=prompt)
    output = chain.run(input)
    print(output)
    # 定义正则表达式模式
    pattern = r'\[answer\]\s*(.*?)\s*(?=\[|\Z)'

    match = re.search(pattern, output, re.DOTALL)
    
    return match

In [ ]:
def answer(query,context):
    prompt = PromptTemplate(
    input_variables=["query, contexts"],
    template="""
    你是一个智能体助手，我给你一个query和一些相关的上下文contexts,由于上下文过长，你需要从中提炼出对于解答query相关的，直接的，有用的信息。
    query: {query}
    contexts: {contexts}
    首先你需要对query进行分析，明确query的意图，以及分析出解答query所需要的可能的信息。
    然后需要对contexts进行分析，提炼出所有有用的信息，最终输出一个优化的上下文，用来解答query。输出的结果需要包含三元组和context，确保这些三元组和context是与query相关的。
    你需要输出三元组及其对三元组的解释，以及context。
    你只需要提炼出与query相关的信息！！！尽量保持原文信息一致性，不用描述和概括。
    不允许出现英文回答，只能包含英文。
    你的输出必须符合下述格式！不允许输出别的内容。
    请一句一句分析，不能跳过任何一句，一步一步思考。
    输出格式如下：
    Triplets:
    (node1,relation,node2)，后面是对形成三元组和解答query有用的信息的解释。
    Context: 
    优化提炼之后的上下文。
    请开始提炼:
    """
    )
    input = {
        'query': query,
        'contexts': context
    }
    chain = LLMChain(llm=llm, prompt=prompt)
    output = chain.run(input)
    print(output)

In [ ]:
from langchain.prompts import ChatPromptTemplate
import pandas as pd
df = pd.read_csv("dataset.csv")
query= df['question']

求rag系统平均时间

In [ ]:
from langchain.prompts import ChatPromptTemplate
# Combine all contexts into a single string
#final_context = "\n\n".join(all_contexts)
#print(final_context)

# Create prompt template
PROMPT_TEMPLATE = """
根据以下提供的Triplets,context的信息来回答问题：
information:{context}
根据上述上下文回答问题：
query：{question}。
首先分析问题，明确解答问题的信息，然后在提供的information中寻找答案，请一步一步思考。
答案不要分段和分点！！！
必须完整的回答问题，包括query中可能的多个问题！
答案必须和query高度相关！确保是能够回答问题的最直接、准确、明了的答案。
不要为你的答案提供理由！！！
不要提供上下文信息中未提到的信息，不要胡编乱造。
请提供与解决问题最直接相关的答案，不需要推理过程！！！
不要说“根据上下文”或“在上下文中提到”或类似的话。
你只需要生成回答的答案，不能出现别的无关的语句！！！
"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

In [ ]:
import time
total_time = 0
query_count = 0
for q in query:
    start_time = time.time() 
    sub_q = rewrite_and_split_query(q)
    triplex,chunks = query_database(sub_q)
    triples = "\n".join(triplex)
    text = ''
    for content in chunks:
        text += str(content)
    combined_context = f"Triplets:\n{triplex}\n\nContext:\n{text}"
    final_context = optimize_contexts(q, combined_context)
    prompt = prompt_template.format(context=final_context, question=q)
    model = llm 
    response_text = model.predict(prompt)
    end_time = time.time()  # 记录结束时间
    elapsed_time = end_time - start_time  # 计算单次查询的时间
    total_time += elapsed_time
    query_count += 1

    # print(f"Query: {q}")
    # print(f"Response: {response_text}")
    print(f"Time taken: {elapsed_time:.4f} seconds\n")

average_time = total_time / query_count  # 计算平均时间
print(f"Average time taken per query: {average_time:.4f} seconds")
    


In [ ]:
results = []
all_contexts = []

# Loop through each query, retrieve context, and store it
for q in query:
    sub_querys =rewrite_and_split_query(q)
    tripes, chunks = query_database(sub_querys)
    triples = "\n".join(tripes)
    text = ''
    for content in chunks:
        text += str(content)
    combined_context = f"Triplets:\n{tripes}\n\nContext:\n{text}"
    final_context = optimize_contexts(q, combined_context)
    all_contexts.append(final_context)

In [ ]:
responses = []
for i, q in enumerate(query):
    prompt = prompt_template.format(context=all_contexts[i], question=q)
    model = llm 
    response_text = model.predict(prompt)
    responses.append(response_text)


In [ ]:
df['answer'] = responses

In [ ]:
df['answer'] = responses
df["retrival_contexts"] = all_contexts
print("Existing columns:", df.columns)
df.to_csv("output.csv", index=False)